In [29]:
import numpy as np
import scipy
from scipy.sparse import diags
from sys import getsizeof
from numpy.linalg import multi_dot as md

In [30]:
class ImplicitALS(object):
    def __init__(self, factors=10, iterations=100, lambda_=4, alpha=40):
        self.iterations = iterations
        self.lambda_ = lambda_
        self.alpha = alpha
        self.factors = factors
        self.Yi = None
        self.Xu = None
    
    def fit(self, matrix):
        Pui = (matrix > 0).astype('int')
        Cui = 1 + self.alpha * matrix
        Xu = np.random.normal(0, size=(matrix.shape[0], self.factors))
        Yi = np.random.normal(0, size=(matrix.shape[1], self.factors))
        for _iter in range(self.iterations):
            for user in range(matrix.shape[0]):
                a = md([Yi.T, np.diag(Cui[user]), Yi]) + self.lambda_ * np.eye(self.factors)
                b = md([Yi.T, np.diag(Cui[user]), Pui[user]])
                Xu[user] = np.linalg.solve(a, b)
            
            for item in range(matrix.shape[1]):
                a = md([Xu.T, np.diag(Cui[:, item]), Xu]) + self.lambda_ * np.eye(self.factors)
                b = md([Xu.T, np.diag(Cui[:, item]), Pui[:, item]])
                Yi[item] = np.linalg.solve(a, b)
        self.Xu = Xu
        self.Yi = Yi
        
            
        

In [31]:
a = ImplicitALS()

In [36]:
a.fit(np.eye(10))

In [38]:
a.Xu[0] @ a.Yi[0]

0.9024390243902438

In [42]:
a.Xu[0] @ a.Yi[4]

1.457167719820518e-16